# Notebook Setup

In [ ]:
import os
from dotenv import load_dotenv

import ep_parse.core as core
import ep_parse.case_data as cdata
import ep_parse.nb_utils.click_registering as crg
import ep_parse.utils as pu
import ep_parse.nb_utils.signal_display as sd
import ep_parse.nb_utils.common as nbu

load_dotenv()
pu.configure_logging()

%load_ext autoreload
%autoreload 2

case_id = "CASE001"

# Clean Export Files from the EPMED system
Warning - Destructive Operations ... be sure to backup your raw exports

In [ ]:
import ep_parse.epmed.file_cleanup as cln

CASE_BIN_DIRECTORY = f"data/{case_id}/epmed"

In [ ]:
# Rename BIN files to remove inconsistent spaces and capitalization
cln.rename_BINS(CASE_BIN_DIRECTORY, False)

In [ ]:
# Remove redundant Page files (just reduce storage size for these raw files)
cln.remove_extra_page_files(CASE_BIN_DIRECTORY, qa_check=False)

In [ ]:
# Remove data TXT files
cln.remove_TXT_datafiles(CASE_BIN_DIRECTORY, True)

In [ ]:
# Remove name row from session text files ... if applicable
cln.remove_pii(CASE_BIN_DIRECTORY, case_id)

In [ ]:
### Remove duplicate overlapping exports
to_remove = cln.check_partial_export(CASE_BIN_DIRECTORY, assert_if=False)
for f in to_remove:
    os.remove(os.path.join(CASE_BIN_DIRECTORY, f))

# P1 Import Data Into System

## P1.0 Create Meta and Events files and view Data Coverage

In [ ]:
cdata.create_case_meta(case_id)

In [ ]:
core.plot_raw_data_coverage(case_id)

## P1.1 Load Signal Data into System
Clean and transform signal data that was exported from the research center's epsystem and write it to a `signals.h5` file.<br>
**Prereqs**
- Exported signal data conforms to the standardized layout (see README) and is located in a folder named by the case_id (e.g. folder name is `CASE001`) in the `os.environ.get("data_filepath")` directory.


**Output**
- An h5 file (`{case_id}_signals.h5`) file in the case's root directory.

In [ ]:
to_import = {"events", "signals"}  # subset of {"signals", "events"} specifying which data to parse (events = bookmarks log file)
### End Configs

core.format_and_store_case_export(case_id, only=to_import, compress=False)
cdata.store_pps_in_meta(case_id)

### P1.1.2 Plot undecorated signals

#### P1.1.2.1 Plot by RF

In [ ]:
rfs = "5" #,15,25,35"   # "" -> all RFs, otherwise format is "1,2,3,4-6,12"

plot_configs = {
    "width": 40,  # set plot width
    "height": 10,      # adjust plot height
    "prepend_seconds": 6,
    "append_seconds": 2,
    "y_scale": 2,     # adjust the spacing of traces within a plot
    "suppress_channels": [],  # list channels which should not appear in the plot
}
# 'V2','V3','V4','V5','V6'
# 'CS_9-10','CS_7-8','CS_5-6','CS_3-4','CS_1-2','PA_1-2','PA_3-4','PB_5-6','PB_7-8','PC_9-10','PC_11-12','PD_13-14','PD_15-16','PE_17-18','PE_19-20'
# 'HRA_9-10','HRA_7-8','HRA_5-6','HRA_3-4','HRA_1-2'
# 'PA_1-2','PA_3-4','PB_5-6','PB_7-8','PC_9-10','PC_11-12','PD_13-14','PD_15-16','PE_17-18','PE_19-20'
# 'A1-A2','A2-A3','A3-A4','B1-B2','B2-B3','B3-B4','C1-C2','C2-C3','C3-C4','D1-D2','D2-D3','D3-D4',"A1-B1","A2-B2","A3-B3","B1-C1","B2-C2","B3-C3","C1-D1","C2-D2","C3-D3"

################   END Configs  ################

event_df, bookmark_df = core.load_events(case_id, outputs=['epsys_events', 'epsys_RFs'])

pconfigs = {
    "signals_plot": plot_configs,
    "delineate_by": "RF",
    "delineation_ids": nbu.parse_rf_str(rfs) if rfs else bookmark_df["RF"].tolist(),
}

_ = sd.plot_undecorated_signals(case_id, pconfigs, event_df)

#### P1.1.2.2 Plot by Time

In [ ]:
times = "12:45:10|after|8"   # "11:20:01, 11:22:12|before|12, 11:33:13|after|20"
step = 10

plot_configs = {
    "width": 40,  # adjust plot width
    "height": 10,      # adjust plot height
    "prepend_seconds": 0,
    "append_seconds": 10,
    "y_scale": 2,     # adjust the spacing of traces within a plot
    "suppress_channels": [],  # list channels which should not appear in the plot
}
# 'V2','V3','V4','V5','V6'
# 'CS_9-10','CS_7-8','CS_5-6','CS_3-4','CS_1-2','PA_1-2','PA_3-4','PB_5-6','PB_7-8','PC_9-10','PC_11-12','PD_13-14','PD_15-16','PE_17-18','PE_19-20'
# 'HRA_9-10','HRA_7-8','HRA_5-6','HRA_3-4','HRA_1-2'
# 'PA_1-2','PA_3-4','PB_5-6','PB_7-8','PC_9-10','PC_11-12','PD_13-14','PD_15-16','PE_17-18','PE_19-20'
# 'A1-A2','A2-A3','A3-A4','B1-B2','B2-B3','B3-B4','C1-C2','C2-C3','C3-C4','D1-D2','D2-D3','D3-D4',"A1-B1","A2-B2","A3-B3","B1-C1","B2-C2","B3-C3","C1-D1","C2-D2","C3-D3"

################   END Configs  ################

pconfigs = {
    "signals_plot": plot_configs,
    "delineate_by": "TIME",
    "delineation_ids": nbu.parse_user_times(times, step),
}

_ = sd.plot_undecorated_signals(case_id, pconfigs)